In [84]:
import pandas as pd

In [85]:
import numpy as np

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [87]:
from sklearn.metrics.pairwise import linear_kernel

In [88]:
products = pd.read_csv('products.csv')

In [89]:
products.head()

,id,name,category,price,level_of_study
0,1,Physics Textbook,Books,29.99,Undergraduate
1,2,Advanced Mathematics,Books,39.99,Graduate
2,3,Intro to Programming,Books,19.99,Beginner
3,4,Biology Lab Kit,Lab Equipment,59.99,Undergraduate
4,5,Chemistry Set,Lab Equipment,49.99,High School


In [90]:
tfidf = TfidfVectorizer(stop_words = 'english')
tfidf_matrix = tfidf.fit_transform(products['category'])
cosine_sum = linear_kernel(tfidf_matrix, tfidf_matrix)

In [91]:
indices = pd.Series(products.id.values, index=products['name'].drop_duplicates())
cat = pd.Series(products['category'].values, index=products['name'].drop_duplicates())
name = pd.Series(products['name'].values, index=products.index)
level = pd.Series(products['level_of_study'].values, index=products.index)

In [92]:
indices['Laptop']

9

In [93]:
cat['Laptop']

'Electronics'

In [94]:
def sameCategoryProducts(name, cosine_sum = cosine_sum):
    index = indices[name]
    sim_score = sorted(enumerate(cosine_sum[index]), key=lambda x:x[1], reverse=True)[0:20]
    sim_index = [i[0] for i in sim_score]
    sim_index_filtered = [i for i in sim_index if i != index]
    return sim_score

In [95]:
tfid = TfidfVectorizer(stop_words = 'english')
tfid_matrix = tfidf.fit_transform(products['level_of_study'])
cosine_sumf = linear_kernel(tfid_matrix, tfid_matrix)

In [96]:
def sameLevelProducts(idx,cosine_sumf = cosine_sumf):
    index = indices[idx]
    simL_score = sorted(enumerate(cosine_sumf[index]), key=lambda x:x[1], reverse=True)[0:20]
    simL_index = [i[0] for i in simL_score]
    simL_index_filtered = [i for i in simL_index if i != index]
    return simL_score

In [97]:
# Create separate lists for prioritizing elements with the second element equal to 1.0
def getRecommendation(product):
    array1 = sameLevelProducts(product)
    array2 = sameCategoryProducts(product)
    
    set1 = set(array1)
    set2 = set(array2)
    
    # Identify common elements with second element value 1.0
    common_elements = {item for item in set1 & set2 if item[1] == 1.0}
    
    # Separate lists
    priority_common = []
    priority_elements = []
    other_elements = []
    seen = set()
    
    # Function to add unique elements to respective lists
    def add_unique_elements(array):
        for item in array:
            if item[0] not in seen:
                if item in common_elements:
                    priority_common.append(item)
                elif item[1] == 1.0:
                    priority_elements.append(item)
                else:
                    other_elements.append(item)
                seen.add(item[0])
    
    # Add elements from both arrays
    add_unique_elements(array1)
    add_unique_elements(array2)
    
    # Combine the lists: common elements first, then other prioritized elements, then the rest
    combined_array = priority_common + priority_elements + other_elements
    
    # Extract only the first elements
    first_elements = [item[0] for item in combined_array]
    
    return first_elements
    array = [i[0] for i in combined_array]
    return array

In [103]:
recommendation = getRecommendation('Laptop')
print(len(recommendation), recommendation)

33 [9, 18, 28, 37, 46, 47, 5, 8, 17, 19, 26, 27, 29, 35, 38, 44, 48, 53, 54, 55, 56, 63, 69, 76, 84, 92, 99, 0, 1, 2, 3, 4, 6]
